GET VIDEO PROPERTIES AND TRIM, CROP AND ADJUST BRIGHTNESS& CONTRAST

In [69]:
import cv2
import moviepy.editor as mpy
from moviepy.video.fx.all import crop
import ffmpeg
from tqdm import tqdm
import os

In [70]:
# get first frame from video and check video properties
path = 'C:\\Users\\nicol\\NFCyber\\VideoAnalysis\\data\\'
common_name = 'p301'
file_format = '.mp4'

# get all file names in directory into list
files = [file.replace(file_format, '') for file in os.listdir(path) 
             if os.path.isfile(os.path.join(path, file)) and
             common_name in file]
print(f'{len(files)} files found in path directory')

# grab first video in list 
input = path + files[0]
input_file = input + file_format
output = input

# get properties of video
cap = cv2.VideoCapture(input_file)
nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)

# save first frame as _frame0.png
ret, image = cap.read()
cv2.imwrite(f'{output}_frame0.png', image)

# check dimensions of frame
x1, y1 = 0, image.shape[1]
x2, y2 = 0 , image.shape[0]

print(f'dimensions: {image.shape[1]} x {image.shape[0]} px\n'
      f'nframes: {nframes}\n'
      f'fps: {fps}\n'
      f'duration: {nframes/fps} s\n')

1 files found in path directory
dimensions: 1936 x 1216 px
nframes: 4200
fps: 77.0
duration: 54.54545454545455 s



In [4]:
# trim video according to frames
output = ffmpeg.output(ffmpeg.input(input_file).trim(start_frame=0, end_frame=4200), f'{output}_trim.mp4')
ffmpeg.run(output)

(None, None)

In [71]:
# crop frame to find out right parameters
x1, y1 = 320, 0
x2, y2 = 1550 , 1216

frame0 = cv2.imread(f'{input}_frame0.png')
cropped_frame0 = frame0[y1:y2, x1:x2]

cv2.imwrite(f'{output}_frame0_cropped.png', cropped_frame0)
cv2.imshow(f'{output}_cropped', cropped_frame0)
cv2.waitKey(0)

-1

In [72]:
# adjust brightness & contrast of frame
alpha = 1   # brightness: 1.0-original, <1.0-darker, >1.0-brighter
beta = -10    # contrast: 0-unchanged, <0-lower contrast, >0-higher contrast

frame0 = cv2.imread(f'{input}_frame0.png')
edited_frame0 = cv2.convertScaleAbs(frame0, alpha=alpha, beta=beta)

cv2.imwrite(f'{output}_frame0_adjusted.png', edited_frame0)
cv2.imshow(f'{output}_adjusted', edited_frame0)
cv2.waitKey(0)


-1

In [73]:
def adjust_video(input_file):
    cap = cv2.VideoCapture(input_file)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(f'{output}_adjusted.mp4', fourcc, fps, (x2-x1, y2-y1)) 

    for curr_frame in tqdm(range(total_frames)):    # total_frames for whole video
        ret, frame = cap.read()
        if ret:
            frame = frame[y1:y2, x1:x2] # crop frame
            frame = cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)  # adjust brightness & contrast of frame
            video.write(frame)  # add frame to video
        else:
            break

    cv2.destroyAllWindows()
    video.release()

    print(f'{input} done!\ninput: {total_frames} frames, output: {curr_frame+1} frames\n\n')


adjust_video(input_file)

  0%|          | 0/4200 [00:00<?, ?it/s]

100%|██████████| 4200/4200 [02:50<00:00, 24.67it/s]

C:\Users\nicol\NFCyber\VideoAnalysis\data\292_p301s_trim done!
input: 4200 frames, output: 4200 frames


